# Phân loại có sự cố hay không dựa trên data và dự đoán X tiếng sau đó

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [6]:
df = pd.read_csv("clean_feature.csv")
df = df.dropna()
df = df.drop_duplicates()
df["datetime"] = pd.to_datetime(df["Day"]) + pd.to_timedelta(df["hour"], unit="h")
print(df["ServiceID"].unique().shape)  


(12,)


In [7]:
def query_interval(df, t_datetime, T):
    start = t_datetime - pd.Timedelta(hours=T)
    return df[(df["datetime"] >= start) & (df["datetime"] <= t_datetime)]


In [8]:
#query
t = pd.to_datetime("2023-07-27 18:00:00")
result = query_interval(df, t, 12)
result = result.sort_index()
print(result.shape)

(693, 9)


In [9]:
#groupby
features = result.groupby(["datetime", "ServiceID"]).agg(
        total_calls=("count", "sum"), # a> Tổng số lần gọi
        total_success=("passed", "sum"), # b> Tổng số lần gọi thành công
        mean_exec_time=("period", "mean"), # c> Trị trung bình thời gian thực thi
        mean_data_usage=("data", "mean") # d> Trị trung bình dữ liệu sử dụng
    )
matrix = features.unstack(level="ServiceID", fill_value=0)
matrix = matrix.sort_index(axis=1, level=1)
matrix.to_csv("feature.csv", index=False)
matrix

,mean_data_usage,mean_exec_time,total_calls,total_success,mean_data_usage,mean_exec_time,total_calls,total_success,mean_data_usage,mean_exec_time,...,total_calls,total_success,mean_data_usage,mean_exec_time,total_calls,total_success,mean_data_usage,mean_exec_time,total_calls,total_success
ServiceID,0,0,0,0,1,1,1,1,2,2,...,8,8,10,10,10,10,11,11,11,11
datetime,,,,,,,,,,,,,,,,,,,,,
2023-07-27 06:00:00,0.000000,0.0,0,0,623.500000,62.0,2,2,582.241510,63.853785,...,7,6,378355.325000,0.0,120,120,873.046977,63.000000,28155,28155
2023-07-27 07:00:00,0.000000,0.0,0,0,493.989595,62.0,187,187,601.752269,63.949111,...,5,5,378355.395000,0.0,120,120,928.508802,63.000000,46135,46135
2023-07-27 08:00:00,1539.777778,63.0,7,7,462.861153,62.0,577,577,624.835269,63.999436,...,6,6,378354.830575,0.0,120,120,929.086420,63.000000,69951,69951
2023-07-27 09:00:00,1517.906250,63.0,22,22,556.287882,62.0,433,433,621.066943,63.950774,...,17,17,378354.691505,0.0,120,120,942.409857,63.000000,83892,83892
2023-07-27 10:00:00,1890.900000,63.0,10,10,426.753175,62.0,1658,1658,619.292998,63.956362,...,31,31,378354.795525,0.0,120,120,958.386481,63.000000,89333,89333
2023-07-27 11:00:00,1143.800000,63.0,11,11,538.484222,62.0,182,182,623.067411,63.952872,...,18,18,378354.854440,0.0,120,120,937.626937,63.000000,82455,82455
2023-07-27 12:00:00,1891.666667,63.0,4,4,513.886310,62.0,30,30,611.088263,63.949545,...,4,4,378354.920280,0.0,120,120,917.772682,63.000000,70621,70621
2023-07-27 13:00:00,2153.250000,63.0,4,4,588.948204,62.0,343,343,615.677958,63.966780,...,22,22,378354.831101,0.0,120,120,931.690648,62.999985,74829,74829


In [10]:
def label(matrix):
    same_all = (matrix["total_calls"] == matrix["total_success"]).all(axis=1)  
    matrix["label"] = same_all.astype(int)
    print(matrix["label"].value_counts())
    matrix.to_csv("feature.csv", index=False) 
    return matrix

def Split_data(df):
    X = df.drop(columns=["label"])  
    y = df["label"]  
    X
    print("Features shape:", X.shape)
    print("Labels shape:", y.shape)

    n_samples = len(X)
    test_size = 0.2

    test_split = int(n_samples * (1 - test_size))
    X_train_full, X_test = X.iloc[:test_split], X.iloc[test_split:]
    y_train_full, y_test = y.iloc[:test_split], y.iloc[test_split:]

    # Chia tập train thành train và validation
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_full, y_train_full, test_size=0.2, random_state=42, shuffle=True
    )
    print("Train:", X_train.shape, "Validation:", X_val.shape, "Test:", X_test.shape)
    return X_train, X_val, X_test, y_train, y_val, y_test

In [11]:
matrix = label(matrix)
X_train, X_val, X_test, y_train, y_val, y_test = Split_data(matrix)
print("Matrix shape:", matrix.shape)
matrix = matrix.drop(columns=["label"])
matrix

label
1    9
0    4
Name: count, dtype: int64
Features shape: (13, 44)
Labels shape: (13,)
Train: (8, 44) Validation: (2, 44) Test: (3, 44)
Matrix shape: (13, 45)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_15840\7725881.py:9: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  X = df.drop(columns=["label"])
C:\Users\ASUS\AppData\Local\Temp\ipykernel_15840\3370569841.py:4: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  matrix = matrix.drop(columns=["label"])


,mean_data_usage,mean_exec_time,total_calls,total_success,mean_data_usage,mean_exec_time,total_calls,total_success,mean_data_usage,mean_exec_time,...,total_calls,total_success,mean_data_usage,mean_exec_time,total_calls,total_success,mean_data_usage,mean_exec_time,total_calls,total_success
ServiceID,0,0,0,0,1,1,1,1,2,2,...,8,8,10,10,10,10,11,11,11,11
datetime,,,,,,,,,,,,,,,,,,,,,
2023-07-27 06:00:00,0.000000,0.0,0,0,623.500000,62.0,2,2,582.241510,63.853785,...,7,6,378355.325000,0.0,120,120,873.046977,63.000000,28155,28155
2023-07-27 07:00:00,0.000000,0.0,0,0,493.989595,62.0,187,187,601.752269,63.949111,...,5,5,378355.395000,0.0,120,120,928.508802,63.000000,46135,46135
2023-07-27 08:00:00,1539.777778,63.0,7,7,462.861153,62.0,577,577,624.835269,63.999436,...,6,6,378354.830575,0.0,120,120,929.086420,63.000000,69951,69951
2023-07-27 09:00:00,1517.906250,63.0,22,22,556.287882,62.0,433,433,621.066943,63.950774,...,17,17,378354.691505,0.0,120,120,942.409857,63.000000,83892,83892
2023-07-27 10:00:00,1890.900000,63.0,10,10,426.753175,62.0,1658,1658,619.292998,63.956362,...,31,31,378354.795525,0.0,120,120,958.386481,63.000000,89333,89333
2023-07-27 11:00:00,1143.800000,63.0,11,11,538.484222,62.0,182,182,623.067411,63.952872,...,18,18,378354.854440,0.0,120,120,937.626937,63.000000,82455,82455
2023-07-27 12:00:00,1891.666667,63.0,4,4,513.886310,62.0,30,30,611.088263,63.949545,...,4,4,378354.920280,0.0,120,120,917.772682,63.000000,70621,70621
2023-07-27 13:00:00,2153.250000,63.0,4,4,588.948204,62.0,343,343,615.677958,63.966780,...,22,22,378354.831101,0.0,120,120,931.690648,62.999985,74829,74829


## Format mẫu
- Câu 1: Ứng với một thời gian cho trước, hãy rút trích đặc trưng cho mẫu….
Tổ chức thành chương trình con 
- Câu 2: Viết chương trình chuẩn bị bộ dữ liệu M hàng và N cột, trong đó Các cột là các đặc trưng như sau:
- Câu 3: Chia dữ liệu thành tập train/val/ test theo quy tắc …..
- Câu 4: Các đặc trưng sẽ được xử lý theo quy tắc …. . Hãy xây dựng chương trình định nghĩa đối tượng CoumnTransfor để thực hiện viẹc tiền xử lý dữ liệu
- Câu 5: Thực hiện phân loại với Pipeline định nghĩa như sau. Hãy xác định những tham số điều khiển nào cần tinh chỉnh
- Câu 6: Huấn luyện và lựa chọn mô hình tốt nhất bằng Grid search từ dữ liệu train / val
- Câu 7: Đánh giá mô hình dựa vào dữ liệu test
## Đề mẫu
- Câu 1: Cho thời điểm t (tính theo đơn vị giờ), hãy truy vấn các dữ liệu trong khoảng thời gian từ t - T đến t, trong đó T là giá trị đã biết, tính bằng giờ.
- Câu 2: Giả sử M là số thời điểm (t) có thể lấy được trong bộ dữ liệu và N là số dịch vụ, hãy tạo một ma trận có kích thước M * (4 * N) chứa các đặc trưng đã rút trích ở câu 5 cho mỗi dịch vụ: 
    5. Rút Trích Các Đặc Trưng Từ Mỗi Nhóm
    Với mỗi nhóm dịch vụ, rút trích các đặc trưng sau:
    
    a. Tổng số lần gọi (Total Calls)
    
    b. Tổng số lần gọi thành công (Successful Calls)
    
    c. Trị trung bình thời gian thực thi (Average Execution Time)
    
    d. Trị trung bình dữ liệu sử dụng (Average Data Usage)
    
    datetime chia hour và day

- Câu 3: Chia dữ liệu thành tập train/val/ test theo quy tắc (nếu dự đoán phải dùng time series split, ngược lại train test split). Từ việc xây dựng chương trình định nghĩa đối tượng CoumnTransfor để thực hiện viẹc tiền xử lý dữ liệu với việc embeding hour theo (x,y, góc theo giờ như trên đồng hồ) còn day sẽ dùng one hot encoder 

- Câu 5: Thực hiện phân loại với Pipeline định nghĩa như sau. Hãy xác định những tham số điều khiển nào cần tinh chỉnh (học giám sát)
- Câu 6: Huấn luyện và lựa chọn mô hình tốt nhất bằng Grid search từ dữ liệu train / val
- Câu 7: Đánh giá mô hình dựa vào dữ liệu test


# SVC

In [ ]:

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

print("Unique classes:", np.unique(y_train))
print("Counts:", np.bincount(y_train))

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", SVC())
])

param_grid = {
    'svm__C': [0.1, 1, 10],
    'svm__gamma': [0.01, 0.1, 1],
    'svm__kernel': ['rbf', 'linear']
}

grid = GridSearchCV(pipe, param_grid, cv=2) #cross validation chia nhỏ
grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)
print("Best score:", grid.best_score_)
print(grid.best_estimator_)

best_pipe = grid.best_estimator_

best_pipe.fit(X_train, y_train)

Unique classes: [0 1]
Counts: [2 6]
Best params: {'svm__C': 0.1, 'svm__gamma': 0.01, 'svm__kernel': 'rbf'}
Best score: 0.75
Pipeline(steps=[('scaler', StandardScaler()), ('svm', SVC(C=0.1, gamma=0.01))])


,steps,"[('scaler', ...), ('svm', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,C,0.1
,kernel,'rbf'
,degree,3
,gamma,0.01


In [13]:
y_pred = best_pipe.predict(X_test)
print("Test score:", best_pipe.score(X_test, y_test))
print("Thực tế:", y_test.iloc[:5].values)
print("Dự đoán:", y_pred[:5])

Test score: 0.3333333333333333
Thực tế: [0 0 1]
Dự đoán: [1 1 1]


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Đánh giá mô hình
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Báo cáo phân loại:\n", report)
print(conf_matrix)


Báo cáo phân loại:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.33      1.00      0.50         1

    accuracy                           0.33         3
   macro avg       0.17      0.50      0.25         3
weighted avg       0.11      0.33      0.17         3

[[0 2]
 [0 1]]


C:\Users\ASUS\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\ASUS\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\ASUS\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize

## Dự báo t giờ tiếp theo có lỗi hay không: có lỗi (label=0) hay không (label=1).

In [15]:
def make_features(df_raw: pd.DataFrame) -> pd.DataFrame:
    feats = df_raw.groupby(["datetime", "ServiceID"]).agg(
        total_calls=("count", "sum"),          # a) tổng số lần gọi
        total_success=("passed", "sum"),       # b) tổng gọi thành công
        mean_exec_time=("period", "mean"),     # c) mean thời gian thực thi
        mean_data_usage=("data", "mean")       # d) mean dữ liệu sử dụng
    )

    feats["success_rate"] = np.where(
        feats["total_calls"] > 0,
        feats["total_success"] / feats["total_calls"], 0.0
    )
    feats["fail_calls"] = feats["total_calls"] - feats["total_success"]
    wide = feats.unstack(level="ServiceID", fill_value=0)

    # Sắp cột cho ổn định
    wide = wide.sort_index(axis=1, level=0)
    wide = wide.sort_index(axis=0)  # theo thời gian
    return wide


In [16]:
# Dự đoán t giờ tiếp theo
time_predict = 2
t_last = result["datetime"].max()
print(t_last)
df_new = query_interval(df, t_last,time_predict)
print(df_new.shape)

2023-07-27 18:00:00
(150, 9)


In [17]:
X_future_full = make_features(df_new)
X_future = X_future_full.drop(columns=["label"], errors='ignore')  # Loại bỏ cột label nếu có
print("Future features shape:", X_future.shape)
train_cols = X_train.columns
for col in train_cols:
    if col not in X_future_full.columns:
        X_future_full[col] = 0
X_future = X_future_full[train_cols].copy()
y_future_pred = best_pipe.predict(X_future)
y_future_proba = None
try:
    y_future_proba = best_pipe.predict_proba(X_future).max(axis=1) # Lấy xác suất cao nhất
except Exception:
    pass  
pred_df = pd.DataFrame({
        "datetime": X_future.index,         
        "y_pred": y_future_pred
    }).set_index("datetime")

if y_future_proba is not None:
    pred_df["confidence"] = y_future_proba

print("=== Dự đoán t giờ kế tiếp ===")
print(pred_df)

Future features shape: (3, 66)
=== Dự đoán t giờ kế tiếp ===
                     y_pred
datetime                   
2023-07-27 16:00:00       1
2023-07-27 17:00:00       1
2023-07-27 18:00:00       1


# Logistic Regression

In [18]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Pipeline Logistic Regression
pipe = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),  
    ("logreg", LogisticRegression(
        class_weight="balanced",
        max_iter=5000,
        solver="saga",    
        random_state=42,
    ))
])

# Grid tham số cho Logistic Regression
param_grid = {
    "logreg__penalty": ["l2", "l1"],   
    "logreg__C": [0.1, 1, 10]
}

# Cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=cv,
    scoring="balanced_accuracy",  # có thể đổi sang accuracy hoặc balanced_accuracy
    verbose=1
)

grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)
print("Best CV score (balanced_accuracy):", grid.best_score_)

best_pipe = grid.best_estimator_

# Đánh giá trên test
y_pred = best_pipe.predict(X_test)
print("\n==== TEST METRICS ====")
print(classification_report(y_test, y_pred, digits=4))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))

# ================== DỰ ĐOÁN t GIỜ TIẾP THEO ==================
time_predict = 2
t_last = result["datetime"].max()
print("t_last:", t_last)

df_future_raw = query_interval(df, t_last, time_predict)
print("df_future_raw shape:", df_future_raw.shape)

if df_future_raw.empty:
    print("⚠️ Không có data thật trong khoảng dự đoán.")
else:
    X_future_full = make_features(df_future_raw)

    # Đồng bộ cột với X_train
    train_cols = X_train.columns
    for col in train_cols:
        if col not in X_future_full.columns:
            X_future_full[col] = 0
    X_future = X_future_full[train_cols].copy()

    # Dự đoán
    y_future = best_pipe.predict(X_future)
    try:
        conf = best_pipe.predict_proba(X_future).max(axis=1)
    except Exception:
        conf = None

    pred_df = pd.DataFrame({"datetime": X_future.index, "y_pred": y_future}).set_index("datetime")
    if conf is not None:
        pred_df["confidence"] = conf
    print("\n=== Dự đoán t giờ tiếp theo ===")
    print(pred_df)


Fitting 5 folds for each of 6 candidates, totalling 30 fits


C:\Users\ASUS\AppData\Roaming\Python\Python313\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\ASUS\AppData\Roaming\Python\Python313\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\ASUS\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:2801: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
C:\Users\ASUS\AppData\Roaming\Python\Python313\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\ASUS\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:2801: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains 

Best params: {'logreg__C': 0.1, 'logreg__penalty': 'l1'}
Best CV score (balanced_accuracy): 0.8

==== TEST METRICS ====
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000         2
           1     0.3333    1.0000    0.5000         1

    accuracy                         0.3333         3
   macro avg     0.1667    0.5000    0.2500         3
weighted avg     0.1111    0.3333    0.1667         3

Confusion matrix:
 [[0 2]
 [0 1]]
t_last: 2023-07-27 18:00:00
df_future_raw shape: (150, 9)

=== Dự đoán t giờ tiếp theo ===
                     y_pred  confidence
datetime                               
2023-07-27 16:00:00       1         0.5
2023-07-27 17:00:00       1         0.5
2023-07-27 18:00:00       1         0.5


C:\Users\ASUS\AppData\Roaming\Python\Python313\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\ASUS\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:2801: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
C:\Users\ASUS\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\ASUS\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control th